# Perform Research with AutoGen

In [ ]:
# import libraries

from lib_imports import *
import logging

import autogen
from autogen import UserProxyAgent, AssistantAgent

In [2]:
# load environment variables

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY')



In [3]:
# log calls assigned to agents

logging.basicConfig(
    level=logging.DEBUG
) 

In [4]:
# initialize agentops
agentops.init(api_key=AGENTOPS_API_KEY, default_tags=["research-paper"])

Exception in thread Thread-3 (check_agentops_update):
Traceback (most recent call last):
  File "C:\Users\Olamide T Iselowo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "d:\Artificial Synapse Media\AutoGen_Research_Agents\aiagentsenv1\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\Olamide T Iselowo\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "d:\Artificial Synapse Media\AutoGen_Research_Agents\aiagentsenv1\Lib\site-packages\agentops\helpers.py", line 132, in check_agentops_update
    conn.request("GET", "/pypi/agentops/json")
  File "C:\Users\Olamide T Iselowo\AppData\Local\Programs\Python\Python311\Lib\http\client.py", line 1282, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\Olamide T Iselowo\AppData\Local\Programs\Pyth

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_events HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.co

In [ ]:
# research topic
topic = "Role of AI in Climate Modelling for Environmental Sustainability"

### AutoGen

In [ ]:
# set configurations

config_list_gpt4 = [{"model": "gpt-4o", "api_key": OPENAI_API_KEY}]

gpt4_config = {
    "cache_seed": 42,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list_gpt4,
    "timeout": 120,
}

## Construct Agents

### Agents Using AutoGen

In [ ]:

user_proxy = UserProxyAgent(
    name="Admin",
    system_message="A human admin. Initialize the chat and pass on the instruction to the GenExplainer",
    code_execution_config=False,
)

genexplainer = AssistantAgent(
    name="GenExplainer",
    llm_config=gpt4_config,
    system_message="""GenExplainer. You collect research topic and break it down to subtopics, identifying key research 
                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable 
                    research questions for an academic research. If the research topic is not good enough to generate 
                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. 
                    You output the research topic, key research questions and hypothesis generated from the research topic.
                    """,
)

genreviewer = AssistantAgent(
    name="GenReviewer",
    llm_config=gpt4_config,
    system_message="""GenReviewer. You receive the research topic and subtopics from the GenExplainer Agent and then gather existing 
                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize 
                    these findings, identify research gaps and organize the information. Produce this summary in the 
                    format of a standard academic literature review, highlighting important existing research and 
                    citing papers used in the review.
                    """,
)

genformulator = AssistantAgent(
    name="GenFormulator",
    system_message="""GenFormulator. You refine the research questions and subtopics from the GenExplainer Agent based on the 
                    Literature Review done by the GenReviewer and define the exact question the paper will address. If a 
                    hypothesis is required, propose one. Your output is the finalized research question and hypothesis.
                    """,
    llm_config=gpt4_config,
)

gendirector = AssistantAgent(
    name="GenDirector",
    llm_config=gpt4_config,
    system_message="""GenDirector. You receive the final research question from GenFormulator and literature review from 
                    GenReviewer and suggest appropriate research methods (qualitative, quantitative, or mixed) based on 
                    the research question. Also, outline data collection methods, sample sizes, tools, and techniques 
                    necessary for this research. Your output is the proposed research methodology, proposing necessary 
                    data collection methods, analysis plans, and tools for the research.
                    """,
)

gencollector = AssistantAgent(
    name="GenCollector",
    llm_config=gpt4_config,
    system_message="""GenCollector. You receive the proposed methodology from GenDirector Agent and gather real data from the 
                    web or simulate the data related to the research question. Ensure that the data are from verifiable 
                    and reliable sources, and are linked to the research being conducted. Your output are simulated or 
                    recommended datasets for analysis.
                    """,
)

genanalyst = AssistantAgent(
    name="GenAnalyst",
    system_message="""GenAnalyst. You receive data and analyze the collected data using appropriate statistical or 
                    qualitative analysis methods from GenCollector. For quantitative research, you could run statistical tests; for 
                    qualitative research, you would conduct thematic analysis. Your output are charts, graphs, 
                    insights, detected trends and predictions (if there are any). The analysis should be dynamic, 
                    customized to suit the requirement of the research field lies in.
                    """,
    llm_config=gpt4_config,
)

geninterpreter = AssistantAgent(
    name="GenInterpreter",
    llm_config=gpt4_config,
    system_message="""GenInterpreter. You collect data analysis results from GenAnalyst Agent and literature review from 
                    GenReviewer. You should interpret the analysis results, compares them with existing research in the 
                    review, and provides insights into how the findings answer the research question or support/refute 
                    the hypothesis, provided by the Formulator. Your output is a discussion of findings, implications, 
                    and potential limitations.
                    """,
)

genrecommender = AssistantAgent(
    name="GenRecommender",
    llm_config=gpt4_config,
    system_message="""GenRecommender. You collect discussion of findings, implications and potential limitations from the 
                    GenInterpreter Agent and summarize the research findings, proposes future research directions, and 
                    provides recommendations or solutions based on the findings. Your output is conclusion and 
                    recommendation section of the paper.
                    """,
)

genformatter = AssistantAgent(
    name="GenFormatter",
    system_message="""GenFormatter. You collect different parts of the research paper and ensure the entire research paper 
                    is formatted according to the chosen academic style (APA, MLA, etc.). Generate the fully formatted 
                    research paper using the final research topic from the GenFormulator Agent, the literature review 
                    from the GenReviewer Agent, the methodology from the GenDirector Agent, the data analysis from the 
                    GenAnalyst Agent, the discussion from the GenInterpreter Agent and the recommendation and conclusion
                    from the GenRecommender Agent. You also ensure proper citation process, collecting and formatting 
                    citations from literature sources. Your output is a fully formatted academic paper with citations 
                    and references.
                    """,
    llm_config=gpt4_config,
)

genqaengineer = AssistantAgent(
    name="GenQAEngineer",
    system_message="""GenQAEngineer. You collect a draft of an academic research paper from GenFormatter and perform proofreading, plagiarism 
                    checks, grammar and style analysis, and overall coherence evaluation. Your output is a PDF document 
                    containing a refined, high-quality research paper. 
                    """,
    llm_config=gpt4_config,
)

critic = AssistantAgent(
    name="Critic",
    system_message="Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL.",
    llm_config=gpt4_config,
)

groupchat = autogen.GroupChat(
    agents=[user_proxy, genexplainer, genreviewer, genformulator, gendirector, gencollector, genanalyst, geninterpreter, genrecommender, genformatter, genqaengineer, critic], messages=[], max_round=3
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)

In [ ]:

user_proxy.initiate_chat(
    manager,
    message="""Write an academic research paper on the topic: """ + topic,
)

### Agents using OpenAI and AgentOps

In [5]:
from agent_classes import ExplainerAgent, LiteratureReviewer, FormulationAgent, MethodAgent, CollectorAgent
from agent_classes import DataAnalyst, DiscussionAgent, RecommendationAgent, CitationAgent, QAEngineer

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='d:\\Artificial Synapse Media\\AutoGen_Research_Agents\\aiagentsenv1\\Lib\\site-packages\\certifi\\cacert.pem'


In [6]:
explainer = ExplainerAgent()
reviewer = LiteratureReviewer()
formulator = FormulationAgent()
director = MethodAgent()
collector = CollectorAgent()
analyst = DataAnalyst()
interpreter = DiscussionAgent()
recommender = RecommendationAgent()
formatter = CitationAgent()
qaengineer = QAEngineer()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/create_agent HTTP/11" 200 9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connect

### Research Paper Generation

In [8]:
# Topic Understanding 

sub_topics = explainer.completion(
    "Generate research questions, subtopics and hypothesis for this topic : \n " + topic
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Explainer. You collect research topic and break it down to subtopics, identifying key research \n                    questions or hypothesis. You also analyze the scope of the research topic and generate suitable \n                    research questions for an academic research. If the research topic is not good enough to generate \n                    enough scope for academic research, rephrase the topics and generate the research questions and hypothesis. \n                    You output the research topic, key research questions and hypothesis generated from the research topic.\n                    '}, {'role': 'user', 'content': 'Generate research questions, subtopics and hypothesis for this topic : \n Role of AI in Climate Modelling for Environmental Sustainability'}], 'model': 'gpt-4o', 'temperature': 0.5}}
DEBUG:opena

In [9]:
print(sub_topics)

**Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability

**Scope of the Research Topic:**
The topic explores the integration of artificial intelligence (AI) technologies in climate modelling, focusing on how these advancements can contribute to environmental sustainability. The research encompasses various AI methodologies, their applications in climate science, and the potential benefits and challenges associated with their use in promoting sustainable environmental practices.

**Subtopics:**
1. **Overview of AI Technologies in Climate Science:**
   - Historical development and current state of AI in climate modelling.
   - Comparison of traditional climate modelling techniques with AI-enhanced methods.

2. **AI Techniques and Tools in Climate Modelling:**
   - Machine learning algorithms and their applications in climate data analysis.
   - Use of neural networks and deep learning in predicting climate patterns.

3. **Impact of AI on Climate Prediction Ac

In [10]:
# Literature Review

literature = reviewer.completion(
    "Write a literature review for these research questions: \n " + sub_topics
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Reviewer. You receive the research topic and subtopics from the Explainer Agent and then gather existing \n                    research, journal articles, papers, and other relevant literature from trusted sources. Summarize \n                    these findings, identify research gaps and organize the information. Produce this summary in the \n                    format of a standard academic literature review, highlighting important existing research and \n                    citing papers used in the review.\n                    '}, {'role': 'user', 'content': 'Write a literature review for these research questions: \n **Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe topic explores the integration of artificial intelligence (AI) technologies in cli

In [11]:
# Research Question Formulation

fin_question= formulator.completion(
    "Formulate a more concrete research question and hypothesis using the research topics: \n " + sub_topics + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Formulator. You refine the research questions and subtopics from the Explainer Agent based on the \n                    Literature Review done by the Reviewer and define the exact question the paper will address. If a \n                    hypothesis is required, propose one. Your output is the finalized research question and hypothesis.\n                    '}, {'role': 'user', 'content': 'Formulate a more concrete research question and hypothesis using the research topics: \n **Research Topic:** Role of AI in Climate Modelling for Environmental Sustainability\n\n**Scope of the Research Topic:**\nThe topic explores the integration of artificial intelligence (AI) technologies in climate modelling, focusing on how these advancements can contribute to environmental sustainability. The research encompasses various AI methodologi

In [12]:
# Research Methodology

methods = director.completion(
    "Suggest appropriate research methods using the final research question: \n " + fin_question + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Director. You receive the final research question from Formulator and literature review from \n                    Reviewer and suggest appropriate research methods (qualitative, quantitative, or mixed) based on \n                    the research question. Also, outline data collection methods, sample sizes, tools, and techniques \n                    necessary for this research. Your output is the proposed research methodology, proposing necessary \n                    data collection methods, analysis plans, and tools for the research.\n                    '}, {'role': 'user', 'content': 'Suggest appropriate research methods using the final research question: \n **Finalized Research Question:**\nHow does the integration of artificial intelligence (AI) technologies enhance the accuracy and efficiency of climate models, and w

In [13]:
# Data Collection

data = collector.completion(
    "Gather necessary data using suggested research methodology: \n " + methods 
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Collector. You receive the proposed methodology from Director Agent and gather real data from the \n                    web or simulate the data related to the research question. Ensure that the data are from verifiable \n                    and reliable sources, and are linked to the research being conducted. Your output are simulated or \n                    recommended datasets for analysis.\n                    '}, {'role': 'user', 'content': 'Gather necessary data using suggested research methodology: \n **Proposed Research Methodology**\n\n**Research Design: Mixed Methods Approach**\n\nGiven the complexity of the research question, a mixed methods approach is appropriate. This approach will allow for a comprehensive analysis by combining qualitative insights with quantitative data, addressing both the technical enhancem

In [30]:
print(data)

To implement the proposed research methodology, we need to gather both quantitative and qualitative data. Here are some recommended datasets and strategies for data collection and simulation:

### Quantitative Data

1. **Secondary Data Analysis:**
   - **WeatherBench Dataset:** A benchmark dataset for data-driven weather forecasting, providing historical weather data that can be used to train and evaluate AI models.
     - Source: [WeatherBench GitHub](https://github.com/pangeo-data/WeatherBench)
   - **ECMWF Datasets:** Access datasets from the European Centre for Medium-Range Weather Forecasts, which include comprehensive climate data such as temperature, precipitation, and atmospheric pressure.
     - Source: [ECMWF Public Datasets](https://www.ecmwf.int/en/forecasts/datasets)
   - **NASA Earth Data:** Utilize data from NASA's Earth Observing System Data and Information System (EOSDIS) for global climate variables.
     - Source: [NASA Earth Data](https://earthdata.nasa.gov/)

2. **

In [14]:
# Data Analysis

analysis = analyst.completion(
    "For academic research purposes, analyse this data: \n " + data 
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Analyst. You receive data and analyze the collected data using appropriate statistical or \n                    qualitative analysis methods. For quantitative research, you could run statistical tests; for \n                    qualitative research, you would conduct thematic analysis. Your output are charts, graphs, \n                    insights, detected trends and predictions (if there are any). The analysis should be dynamic, \n                    customized to suit the requirement of the research field lies in.\n                    '}, {'role': 'user', 'content': "For academic research purposes, analyse this data: \n To implement the proposed research methodology, we need to gather both quantitative and qualitative data. Here are some recommended datasets and strategies for data collection and simulation:\n\n### Quantit

In [15]:
# Discussion Phase

discourse = interpreter.completion(
    "Write a discussion of findings and possible implications using the data analysis: \n " + analysis + "\n  and the literature review: \n" + literature
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Interpreter. You collect data analysis results from Analyst Agent and literature review from \n                    Reviewer. You should interpret the analysis results, compares them with existing research in the \n                    review, and provides insights into how the findings answer the research question or support/refute \n                    the hypothesis, provided by the Formulator. Your output is a discussion of findings, implications, \n                    and potential limitations.\n                    '}, {'role': 'user', 'content': "Write a discussion of findings and possible implications using the data analysis: \n ### Quantitative Analysis\n\n#### 1. Data Collection and Preparation:\n- **WeatherBench Dataset, ECMWF Datasets, and NASA Earth Data** will be utilized to gather historical climate data. These da

In [16]:
# Conclusion and Recommendations

conclusion = recommender.completion(
    "Write a recommendation and coclusion using the discussion generated from the academic research: \n " + discourse
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Recommender. You collect discussion of findings, implications and potential limitations from the \n                    Interpreter Agent and summarize the research findings, proposes future research directions, and \n                    provides recommendations or solutions based on the findings. Your output is conclusion and \n                    recommendation section of the paper.\n                    '}, {'role': 'user', 'content': "Write a recommendation and coclusion using the discussion generated from the academic research: \n ### Discussion of Findings\n\nThe integration of AI into climate modeling, as explored through our quantitative and qualitative analyses, aligns with the growing body of literature that highlights AI's transformative impact on climate science. Our findings demonstrate that AI-enhanced models outp

In [17]:
# Formatting and Citation

draft =formatter.completion(
    "Generate the fully formatted research paper using the final research topic: \n " + fin_question + "\n the literature review:" + literature
    + "\n the methodology:" + methods + "\n the data analysis:" + analysis + "\n the discussion:" + discourse + "  \n and the recommendation and conclusion:" + conclusion 
)
 


DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'Formatter. You collect different parts of the research paper and ensure the entire research paper \n                    is formatted according to the chosen academic style (APA, MLA, etc.). You also ensure proper \n                    citation process, collecting and formatting citations from literature sources. Your output is a \n                    fully formatted academic paper with citations and references.\n                    '}, {'role': 'user', 'content': "Generate the fully formatted research paper using the final research topic: \n **Finalized Research Question:**\nHow does the integration of artificial intelligence (AI) technologies enhance the accuracy and efficiency of climate models, and what are the implications for promoting environmental sustainability through improved resource management and policy-making?\n

In [18]:
# Complete Research Paper

paper =qaengineer.completion(
    "Do the final touch up of the draft: \n " + draft + "output a PDF format of the doocument"
)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'QAEngineer. You collect a draft of an academic research paper and perform proofreading, plagiarism \n                    checks, grammar and style analysis, and overall coherence evaluation. Your output is a PDF document \n                    containing a refined, high-quality research paper. \n                    '}, {'role': 'user', 'content': "Do the final touch up of the draft: \n # Integration of Artificial Intelligence in Climate Modelling: Enhancing Accuracy and Efficiency for Environmental Sustainability\n\n## Abstract\n\nThe integration of artificial intelligence (AI) technologies, particularly machine learning and deep learning, into climate modeling has the potential to significantly enhance the accuracy and efficiency of climate predictions. This paper explores how AI technologies improve climate models and the im

In [28]:
with open("climatemodelling.md", "w") as file: file.write(paper)

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def save_to_pdf(text, filename):
    c = canvas.Canvas(filename, pagesize=letter)
    width, height = letter
    for line in text.strip().splitlines():
        height -= 11
        c.drawString(10, height, line.strip())
    c.save()

save_to_pdf(paper, "research.pdf")

In [29]:
agentops.end_session("Success")

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.agentops.ai:443
DEBUG:urllib3.connectionpool:https://api.agentops.ai:443 "POST /v2/update_session HTTP/11" 200 139
🖇 AgentOps: Session Stats - Duration: 3h 9m 32.3s | Cost: $1.295740 | LLMs: 24 | Tools: 0 | Actions: 56 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=a21855e9-396b-45a2-8775-49154cb0829c
